In [45]:
import numpy as np
import keras
import glob
import os
import random
import matplotlib.pyplot as plt
import datetime
import cv2
%matplotlib inline

In [46]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True) 

Mounted at /content/drive


In [47]:
## Generating data

img_width = 224
img_height = 224
img_channel = 3
batch_size = 40


In [48]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [49]:
train_datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)



In [50]:
train_generator = train_datagen.flow_from_directory(directory='/content/drive/My Drive/Colab Notebooks/catdog_dataset/training_set',
                                                    class_mode='categorical',
                                                    color_mode='rgb',
                                                    batch_size=40,
                                                    #seed=30,
                                                    target_size=(224,224))

test_generator = test_datagen.flow_from_directory(directory='/content/drive/My Drive/Colab Notebooks/catdog_dataset/test_set',
                                                    class_mode='categorical',
                                                    color_mode='rgb',
                                                    batch_size=40,
                                                    #seed=30,
                                                    target_size=(224,224))

  

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [51]:
import keras
from keras.models import Sequential , Model
from keras.layers import Dense, Dropout, Activation, Flatten ,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.applications.vgg16 import VGG16
#from keras.applications import ResNet50

In [52]:
num_classes = 2
num_epochs = 30

In [53]:

vgg_model = VGG16(weights='imagenet',include_top=False,input_shape=(img_height,img_width,img_channel))


for layer in vgg_model.layers:
  layer.trainable = False

x = Flatten()(vgg_model.output)
x = Dense(32,activation='relu')(x)
x = Dropout(0.25)(x)
#x = Dense(8,activation='relu')(x)
#x = Dropout(0.5)(x)
x = Dense(2,activation='softmax')(x)

model = Model(inputs=vgg_model.input,outputs=x)

#opt = optimizers.SGD(learning_rate=0.0015,momentum=0.9,nesterov=True)
opt = optimizers.SGD(learning_rate=0.0010)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [54]:
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001) # write the REducelronplateau code here

curr_dt_time = datetime.datetime.now()
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
  os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

callbacks_list = [checkpoint,LR]

In [55]:
#model.fit(train_generator, steps_per_epoch=steps_per_epoch,epochs=num_epochs,verbose=1, callbacks=callbacks_list,validation_data=test_generator,validation_steps=validation_steps)
model.fit(train_generator, epochs=num_epochs,verbose=1, callbacks=callbacks_list,validation_data=test_generator)

Epoch 1/30
200/200 [==============================] - ETA: 0s - loss: 0.6740 - categorical_accuracy: 0.5856
Epoch 00001: val_loss improved from inf to 0.59262, saving model to model_init_2020-08-0919_00_46.179217/model-00001-0.67398-0.58562-0.59262-0.76900.h5
200/200 [==============================] - 151s 756ms/step - loss: 0.6740 - categorical_accuracy: 0.5856 - val_loss: 0.5926 - val_categorical_accuracy: 0.7690
Epoch 2/30
200/200 [==============================] - ETA: 0s - loss: 0.5789 - categorical_accuracy: 0.6976
Epoch 00002: val_loss improved from 0.59262 to 0.48053, saving model to model_init_2020-08-0919_00_46.179217/model-00002-0.57893-0.69762-0.48053-0.78950.h5
200/200 [==============================] - 150s 749ms/step - loss: 0.5789 - categorical_accuracy: 0.6976 - val_loss: 0.4805 - val_categorical_accuracy: 0.7895
Epoch 3/30
200/200 [==============================] - ETA: 0s - loss: 0.5152 - categorical_accuracy: 0.7524
Epoch 00003: val_loss improved from 0.48053 to 0.4